# 10. Variate-covariate models with MCMC

[Data set download](https://s3.amazonaws.com/bebi103.caltech.edu/data/good_invitro_droplet_data.csv)

<hr>

In [1]:
# Colab setup ------------------
import os, shutil, sys, subprocess, urllib.request
if "google.colab" in sys.modules:
    cmd = "pip install --upgrade colorcet datashader bebi103 arviz cmdstanpy watermark"
    process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
    from cmdstanpy.install_cmdstan import latest_version
    cmdstan_version = latest_version()
    cmdstan_url = f"https://github.com/stan-dev/cmdstan/releases/download/v{cmdstan_version}/"
    fname = f"colab-cmdstan-{cmdstan_version}.tgz"
    urllib.request.urlretrieve(cmdstan_url + fname, fname)
    shutil.unpack_archive(fname)
    os.environ["CMDSTAN"] = f"./cmdstan-{cmdstan_version}"
    data_path = "https://s3.amazonaws.com/bebi103.caltech.edu/data/"
else:
    data_path = "../data/"
# ------------------------------

In [2]:
import pandas as pd

import cmdstanpy
import arviz as az

import iqplot
import bebi103

import bokeh.io
bokeh.io.output_notebook()

Loading BokehJS ...

<hr>

In a [previous lesson](../03/plotting_posteriors.ipynb), we plotted posterior distributions of models for microtubule size as a function of the size of droplets encapsulating them using data from [Good, et al., 
 *Science*, **342**, 856-860, 2013](https://doi.org/10.1126/science.1243147). Here, we will perform a similar analysis using MCMC. We start by loading in the data set and making a quick plot.

In [3]:
# Load in Data Frame
df = pd.read_csv(os.path.join(data_path, "good_invitro_droplet_data.csv"), comment="#")

# Pull out numpy arrays
ell = df["Spindle Length (um)"].values
d = df["Droplet Diameter (um)"].values

# Make a plot
p = bokeh.plotting.figure(
    frame_height=200,
    frame_width=300,
    x_axis_label="droplet diameter (µm)",
    y_axis_label="spindle length (µm)",
    x_range=[0, 250],
    y_range=[0, 50],
)

p.circle(
    source=df,
    x="Droplet Diameter (um)",
    y="Spindle Length (um)",
    alpha=0.3,
)

bokeh.io.show(p)

## Updated generative model

We will estimate parameters for the conserved tubulin model. In this model, the spindle length $l$ is a function of droplet diameter $d$ according to

\begin{align}
l(d;\gamma, \phi) = \frac{\gamma d}{\left(1+(\gamma d/\phi)^3\right)^{\frac{1}{3}}}.
\end{align}

We will model the variation off of this curve as Normally distributed (assuming homoscedasticity), such that our likelihood is defined as 

\begin{align}
l_i \mid d_i, \gamma, \phi, \sigma \sim \text{Norm}(l(d_i;\gamma, \phi), \sigma) \;\forall i.
\end{align}

We will choose weakly informative priors. This is a departure from our approach the previous time we studied this data set in which we used Jeffreys priors. The Jeffreys priors allowed for an analytical treatment, but they did not codify our prior knowledge. If we wish to properly encode our prior knowledge in the prior, we usually have to sacrifice analytical tractability. But that's ok, since we have MCMC! 

We will build the prior for the three parameters $\phi$, $\gamma$, and $\sigma$, taking each to be independent of the others. Let's start with $\sigma$. It is possible that the spindle size is very carefully controlled. It is also possible that it could be highly variable. So, we can choose a Half Normal prior for $\sigma$ with a large scale parameter of 10 µm; $\sigma \sim \text{HalfNorm}(10)$. For $\gamma$, we know the physically is must be between zero and one (as we have worked out in previous lessons), so we will assume a prior on is close to uniform that domain, but with probability density dropping right at zero and one; $\gamma \sim \text{Beta}(1.1, 1.1)$. For $\phi$, the typical largest spindle size, we assume a smallest spindle of about a micron (right where I would start to feel uncomfortable betting the farm) and the largest spindle to be about a millimeter (the size of a very large cell, like a *Xenopus* egg). We therefore take $\log_{10}\phi \sim \text{Norm}(1.5, 0.75)$, where $\phi$ is in units of microns.

We thus have our complete generative model.

\begin{align}
&\log_{10} \phi \sim \text{Norm}(1.5, 0.75),\\[1em]
&\gamma \sim \text{Beta}(1.1, 1.1), \\[1em]
&\sigma \sim \text{HalfNorm}(10),\\[1em]
&\mu_i =  \frac{\gamma d_i}{\left(1+(\gamma d_i/\phi)^3\right)^{\frac{1}{3}}}, \\[1em]
&l_i \mid d_i, \gamma, \phi, \sigma \sim \text{Norm}(\mu_i, \sigma) \;\forall i.
\end{align}

## Using Stan to sample

The Stan code we will use is below.

```stan
functions {
  real ell_theor(real d, real phi, real gamma_) {
    real denom_ratio = (gamma_ * d / phi)^3;
    return gamma_ * d / cbrt(1 + denom_ratio); 
  }
}


data {
  int N;
  array[N] real d;
  array[N] real ell;
}


parameters {
  real log10_phi;
  real<lower=0, upper=1> gamma_;
  real<lower=0> sigma;
}


transformed parameters {
  real phi = 10^log10_phi;
}


model {
  log10_phi ~ normal(1.5, 0.75);
  gamma_ ~ beta(1.1, 1.1);
  sigma ~ normal(0.0, 10.0);
  
  for (i in 1:N) {
    ell[i] ~ normal(ell_theor(d[i], phi, gamma_), sigma);
  }
}
```


I pause to point out a few syntactical elements.

1. Note how I used `gamma_` as the variable name for `gamma`. This is because `gamma` is used as a distribution name in Stan.
2. I have used the `functions` block in this Stan program. The definition of the function is `real ell_theor(real d, real phi, real gamma_) { }`, where the code to be executed in the function is between the braces. This is how you declare a function in Stan. The first word, `real` specifies what data type is expected to be returned. Then comes the name of the function, followed by its arguments in parentheses preceded by their data type.
3. Within the function, I had to declare `denom_ratio` to be `real`. Remember, Stan is statically typed, so every variable needs a declaration of its type.
4. Also within the function, note again that the raise-to-power operator is `^`, not `**` as in Python.
5. The `return` statement is like Python; `return` followed by a space and then an expression for the value to be returned.
6. There is no Half-Normal distribution in Stan. It is implemented by putting a bound on the variable (in this case `real<lower=0> sigma;`) and then model `sigma` as being Normally distributed with location parameter zero.
7. I use the `transformed parameters` block to conveniently handle the fact that I wanted to give my prior for $\phi$ in terms of its base-ten logarithm.

All right! Let's do some sampling!

In [4]:
data = dict(
    N=len(df),
    d=df["Droplet Diameter (um)"].values,
    ell=df["Spindle Length (um)"].values,
)

with bebi103.stan.disable_logging():
    sm = cmdstanpy.CmdStanModel(stan_file='spindle.stan')
    
    samples = sm.sample(
        data=data,
        chains=4,
        iter_sampling=1000
    )

samples = az.from_cmdstanpy(samples)

chain 1 |          | 00:00 Status

chain 2 |          | 00:00 Status

chain 3 |          | 00:00 Status

chain 4 |          | 00:00 Status

With samples in hand, let's make a plot of the posterior samples. Remember that when we plotted the posterior by brute force, we had to analytically marginalize out $\sigma$ (which we could do because we used a Jeffreys prior). Here, we can just plot our samples of $\phi$ and $\gamma$ and *ignore* the $\sigma$ samples. So, we can look at a corner plot and check out the $\phi$-$\gamma$ plane.

In [5]:
bokeh.io.show(
    bebi103.viz.corner(
        samples,
        parameters=[("phi", "ϕ"), ("gamma_", "γ"), ("sigma", "σ")],
        plot_ecdf=True,
    )
)

The ECDFs of the samples depicted in the corner plot enable quick reference to the marginalized credible intervals.

We now have a complete picture of the posterior, computed directly from our samples. We will eschew plotting best-fit lines because this is not a preferred method of visualization and defer that to when we do **posterior predictive checks**.

In [6]:
bebi103.stan.clean_cmdstan()

## Computing environment

In [7]:
%load_ext watermark
%watermark -v -p numpy,pandas,cmdstanpy,arviz,bokeh,bebi103,jupyterlab
print("cmdstan   :", bebi103.stan.cmdstan_version())

Python implementation: CPython
Python version       : 3.11.5
IPython version      : 8.15.0

numpy     : 1.26.2
pandas    : 2.1.4
cmdstanpy : 1.2.0
arviz     : 0.17.0
bokeh     : 3.3.0
bebi103   : 0.1.19
jupyterlab: 4.0.10

cmdstan   : 2.33.1
